# Perform a regression on gross and net generation data from multiple years

We want to run a regression on multiple years of CEMS and EIA data. We can do by month, and also on an annual basis

We should probably aggregate by plant-prime mover-environmental equipment.

Steps:
1. Load and clean gross generation data for multiple years
2. Load and distribute net generation data from EIA for multiple years
3. Aggregate / map data from both source

In [2]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

# import the necessary packages
%reload_ext autoreload
%autoreload 2

# import packages
import sqlalchemy as sa
from pathlib import Path
import pandas as pd
import plotly.express as px
import numpy as np
import statsmodels.formula.api as smf
import warnings

# PUDL
import pudl.analysis.allocate_net_gen as allocate_gen_fuel
import pudl.analysis.epa_crosswalk as epa_crosswalk
import pudl.output.pudltabl

# local packages
import src.data_cleaning as data_cleaning
from src.gross_to_net_generation import *
import src.load_data as load_data


In [1]:
year = 2020
number_of_years = 2

start_year = year - (number_of_years - 1)
end_year = year

In [4]:
# load 5 years of monthly data from CEMS and EIA-923
cems_monthly, gen_fuel_allocated = load_monthly_gross_and_net_generation(
    start_year, end_year
)

# add subplant ids to the data
print("Creating subplant IDs")
cems_monthly, gen_fuel_allocated = generate_subplant_ids(
    start_year, end_year, cems_monthly, gen_fuel_allocated
)


loading 2019 CEMS data


a:\miniconda3\envs\hourly_egrid\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


loading 2020 CEMS data


a:\miniconda3\envs\hourly_egrid\lib\site-packages\pandas\core\arrays\datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


Allocating EIA-923 generation data


a:\github\pudl\src\pudl\analysis\allocate_net_gen.py:190: UserWarning: 2 generators are missing prime mover codes in gens_eia860. This will result in incorrect allocation.
  warnings.warn(


Creating subplant IDs


In [5]:
gen_data, plant_aggregation_columns = combine_gross_and_net_generation_data(
    cems_monthly, gen_fuel_allocated, 'subplant'
)

# calculate the hourly average generation values
gen_data["hours_in_month"] = gen_data["report_date"].dt.daysinmonth * 24
gen_data["gross_generation_mw"] = (
    gen_data["gross_generation_mwh"] / gen_data["hours_in_month"]
)
gen_data["net_generation_mw"] = (
    gen_data["net_generation_mwh"] / gen_data["hours_in_month"]
)

gen_data

,plant_id_eia,subplant_id,report_date,gross_generation_mwh,net_generation_mwh,hours_in_month,gross_generation_mw,net_generation_mw
0,3,0.0,2019-03-01,2128.5,68.032782,744,2.860887,0.091442
1,3,0.0,2019-05-01,7656.0,480.035503,744,10.290323,0.645209
2,3,0.0,2019-07-01,4859.0,163.881712,744,6.530914,0.220271
3,3,0.0,2019-09-01,11188.0,1232.974148,720,15.538889,1.712464
4,3,0.0,2019-10-01,3586.5,81.273993,744,4.820565,0.109239
...,...,...,...,...,...,...,...,...
72275,61242,1.0,2019-05-01,NaN,NaN,744,NaN,NaN
72276,61242,1.0,2020-09-01,NaN,3359.802500,720,NaN,4.666392
72277,61242,1.0,2020-10-01,NaN,2905.441500,744,NaN,3.905163
72278,61242,1.0,2020-11-01,NaN,2275.763500,720,NaN,3.160783


In [9]:
# test regression
id = 3
subplant = 4
test_data = gen_data[(gen_data['plant_id_eia'] == id) & (gen_data['subplant_id'] == subplant)]
model = smf.ols('net_generation_mw ~ gross_generation_mw -1', data=test_data).fit()
outliers = model.outlier_test()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:      net_generation_mw   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          1.776e+06
Date:                Thu, 26 May 2022   Prob (F-statistic):                    1.77e-55
Time:                        17:58:59   Log-Likelihood:                         -51.245
No. Observations:                  23   AIC:                                      104.5
Df Residuals:                      22   BIC:                                      105.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
gross_generation_mw     0.6590      0.000   1332.688      0.000       0.658       0.660
==============================================================================
Omnibus:                        5.348   Durbin-Watson:                   1.389
Prob(Omnibus):                  0.069   Jarque-Bera (JB):                3.246
Skew:                          -0.724   Prob(JB):                        0.197
Kurtosis:                       4.137   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
model.params

gross_generation_mw    0.659006
dtype: float64

In [ ]:
# get outputs of final adjusted model
slope = model.params[1]
intercept = model.params[0]
rsquared = model.rsquared
rsquared_adj = model.rsquared_adj
number_observations = model.nobs

In [ ]:
# calculate the ratio for each plant and create a dataframe
gtn_regression = (
    gen_data.dropna().groupby(plant_aggregation_columns).apply(model_gross_to_net)
)

# Load Data
We need to load net generation data from EIA-923 and gross generation data from CEMS

In [ ]:
crosswalk = pudl.output.epacems.epa_crosswalk()

In [ ]:


# TODO: move the following code to a separate function so that it does not hold these dataframes in memory after calculation

# load 5 years of monthly data from CEMS and EIA-923
cems_monthly, gen_fuel_allocated = gross_to_net_generation.load_monthly_gross_and_net_generation(
    start_year, end_year
)

# Test Subplant assignment

In [ ]:
cems_monthly[cems_monthly['plant_id_eia'] == 7063]

In [ ]:
ids = cems_monthly[["plant_id_eia", "unitid"]].drop_duplicates()

In [ ]:
ids[ids['plant_id_eia'] == 7063]

In [ ]:
crosswalk = pudl.output.epacems.epa_crosswalk()

In [ ]:
crosswalk[crosswalk['EIA_PLANT_ID'] == 7063]

In [ ]:
# identify epa unit ids missing from the crosswalk

ids.merge(crosswalk[['EIA_PLANT_ID','CAMD_UNIT_ID']], how='outer', left_on=['plant_id_eia','unitid'], right_on=['EIA_PLANT_ID','CAMD_UNIT_ID'], indicator='source')

In [ ]:
unique_eia_ids = gen_fuel_allocated[
        ["plant_id_eia", "generator_id"]
    ].drop_duplicates()

In [ ]:
unique_eia_ids[unique_eia_ids['plant_id_eia'] == 7063]

# Aggregate the data to the monthly level
For now we will aggregate to plant-month, but in the future we probably want to aggregate at a sub-plant level

## Identify subplants

In [ ]:
subplant_crosswalk = gross_to_net_generation.identify_subplants(start_year, end_year, gen_fuel_allocated, cems_monthly)
# export the crosswalk to csv
subplant_crosswalk.to_csv('../data/outputs/subplant_crosswalk.csv', index=False)
subplant_crosswalk

In [ ]:
# merge the subplant ids into each dataframe
gen_fuel_allocated = gen_fuel_allocated.merge(subplant_crosswalk[['plant_id_eia','generator_id','subplant_id']], how='left', on=['plant_id_eia','generator_id'])
cems_monthly = cems_monthly.merge(subplant_crosswalk[['plant_id_eia','unitid','subplant_id']], how='left', on=['plant_id_eia','unitid'])

## Investigate relationships between units and gens
This is just exploration and not used in this pipeline

In [ ]:
crosswalk = load_data.load_epa_eia_crosswalk(year=2020)[['plant_id_epa','unitid','plant_id_eia','generator_id']]

# drop duplicated rows (might have had multiple entries for boilers, which we do not care about)
crosswalk = crosswalk.drop_duplicates()
# drop rows with missing crosswalks
crosswalk = crosswalk[~crosswalk['plant_id_eia'].isna()]

# one-to-one relationships
one_to_one = crosswalk.copy()[(~crosswalk.duplicated(subset=['plant_id_epa','unitid'], keep=False)) & (~crosswalk.duplicated(subset=['plant_id_eia','generator_id'], keep=False))]
#crosswalk.drop_duplicates(subset=['plant_id_epa','unitid'], keep=False).drop_duplicates(subset=['plant_id_eia','generator_id'], keep=False)
one_to_one['relationship'] = '1:1'

# many-to-one relationships
many_to_one = crosswalk.drop_duplicates(subset=['plant_id_epa','unitid'], keep=False)
many_to_one = many_to_one[many_to_one.duplicated(subset=['plant_id_eia','generator_id'], keep=False)]
many_to_one['relationship'] = 'm:1'

# one_to_many
one_to_many = crosswalk.drop_duplicates(subset=['plant_id_eia','generator_id'], keep=False)
one_to_many = one_to_many[one_to_many.duplicated(subset=['plant_id_epa','unitid'], keep=False)]
one_to_many['relationship'] = '1:m'

# many-to-many
# NOTE: this isn't working entirely correctly
# find all 1:m
many_to_many = crosswalk[crosswalk.duplicated(subset=['plant_id_epa','unitid'], keep=False)]
# create a dataframe that has all epa plant-units in this list that are associated with multiple generators
many_to_many_units = many_to_many[many_to_many.duplicated(subset=['plant_id_eia','generator_id'], keep=False)][['plant_id_epa','unitid']]
many_to_many_units['relationship'] = 'm:m'
# merge this back into the 1:m frame to identify all plant-units that are m:m
many_to_many = many_to_many.merge(many_to_many_units.drop_duplicates(), how='left', on=['plant_id_epa','unitid'])
#remove any rows where the relationship column is na (meaning not m:m)
many_to_many = many_to_many[many_to_many['relationship'] == 'm:m']

relationship = pd.concat([one_to_one,one_to_many,many_to_one,many_to_many], axis=0)

missing = crosswalk.merge(relationship, how='left', on=['plant_id_epa','unitid','plant_id_eia','generator_id'])
missing[missing['relationship'].isna()]

In [ ]:
missing[missing['relationship'].isna()]

## Identify for which months we have complete reporting from all units/generators in a subplant

In [ ]:
# for each subplant-month, get a list of all unitids that reported data in CEMS
missing_units_in_subplant = (
    cems.sort_values(by=["plant_id_eia", "subplant_id", "unitid"])
    .groupby(["plant_id_eia", "subplant_id", "report_date"])["unitid"]
    .unique()
    .apply(list)
    .reset_index()
)
# get a complete list of all of the cems units that belong in each subplant
units_in_subplant = (
    subplant_crosswalk.sort_values(by=["plant_id_eia", "subplant_id", "unitid"])
    .groupby(["plant_id_eia", "subplant_id"])["unitid"]
    .unique()
    .apply(list)
    .reset_index()
)
# merge the list of complete units into the list of monthly reporting units
missing_units_in_subplant = missing_units_in_subplant.merge(
    units_in_subplant,
    how="left",
    on=["plant_id_eia", "subplant_id"],
    suffixes=("_reported", "_complete"),
)
# where is there missing unit reporting
missing_units_in_subplant = missing_units_in_subplant[
    (
        missing_units_in_subplant["unitid_reported"]
        != missing_units_in_subplant["unitid_complete"]
    )
    & (missing_units_in_subplant["unitid_complete"].notnull())
]
missing_units_in_subplant


In [ ]:
# remove unit-months where there is incomplete data
cems = cems.merge(missing_units_in_subplant[['plant_id_eia','subplant_id','report_date']], how='left', indicator=True)
cems = cems[cems['_merge'] != 'both'].drop(columns='_merge')

In [ ]:
# for each subplant-month, get a list of all generator_ids that reported data in CEMS
missing_gens_in_subplant = gen_fuel_allocated.sort_values(by=['plant_id_eia','subplant_id','generator_id']).groupby(['plant_id_eia','subplant_id','report_date'])['generator_id'].unique().apply(list).reset_index()
# get a complete list of all of the cems units that belong in each subplant
units_in_subplant = subplant_crosswalk.sort_values(by=['plant_id_eia','subplant_id','generator_id']).groupby(['plant_id_eia','subplant_id'])['generator_id'].unique().apply(list).reset_index()
# merge the list of complete units into the list of monthly reporting units
missing_gens_in_subplant = missing_gens_in_subplant.merge(units_in_subplant, how='left', on=['plant_id_eia','subplant_id'], suffixes=('_reported','_complete'))
# where is there missing unit reporting
missing_gens_in_subplant = missing_gens_in_subplant[(missing_gens_in_subplant['generator_id_reported'] != missing_gens_in_subplant['generator_id_complete']) & (missing_gens_in_subplant['generator_id_complete'].notnull())]
missing_gens_in_subplant

In [ ]:
gen_fuel_allocated[(gen_fuel_allocated['plant_id_eia'] == 3982) & (gen_fuel_allocated['generator_id'] == '4')]

In [ ]:
# remove generator-months where there is incomplete data
gen_fuel_allocated = gen_fuel_allocated.merge(missing_gens_in_subplant[['plant_id_eia','subplant_id','report_date']], how='left', indicator=True)
gen_fuel_allocated = gen_fuel_allocated[gen_fuel_allocated['_merge'] != 'both'].drop(columns='_merge')

## Aggregate Data

In [ ]:
groupby_columns = ['plant_id_eia','subplant_id','report_date']

net_gen = gen_fuel_allocated.groupby(groupby_columns).sum(min_count=1)['net_generation_mwh'].reset_index()
gross_gen = cems_monthly.groupby(groupby_columns).sum()['gross_generation_mwh'].reset_index()
gen_data = gross_gen.merge(net_gen, how='outer', on=groupby_columns)

# calculate the hourly average generation values
gen_data['hours_in_month'] = gen_data['report_date'].dt.daysinmonth * 24
gen_data['gross_generation_mw'] = gen_data['gross_generation_mwh'] / gen_data['hours_in_month']
gen_data['net_generation_mw'] = gen_data['net_generation_mwh'] / gen_data['hours_in_month']
gen_data

In [ ]:
id = 54
subplant = 3
data_to_plot = gen_data[(gen_data['plant_id_eia'] == id) & (gen_data['subplant_id'] == subplant)]
corner_0 = data_to_plot[['gross_generation_mw','net_generation_mw']].min().min()
corner_1 = data_to_plot[['gross_generation_mw','net_generation_mw']].max().max()

px.scatter(data_to_plot, 
           x='gross_generation_mw', 
           y='net_generation_mw',
           hover_data=['report_date'],
           width=600,
           height=600,
           trendline='ols').add_shape(type="line", x0=corner_0, y0=corner_0, x1=corner_1, y1=corner_1, line=dict(color="Black", width=1))

# Calculate the regressions

In [ ]:
# test regression
id = 3
subplant = 4
test_data = gen_data[(gen_data['plant_id_eia'] == id) & (gen_data['subplant_id'] == subplant)]
model = smf.ols('net_generation_mw ~ gross_generation_mw', data=test_data).fit()
outliers = model.outlier_test()
model.summary()

In [ ]:
def model_gross_to_net(df):
    """
    Performs a linear regression model of monthly gross to net generation.

    Performs recursive outlier removal up to two times if the absolute value of 
    the studentizes residual > 3

    Args:
        df: dataframe containing all values of gross and net generation that should be regressed
    Returns:
        various model parameters
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        # get a linear model for the data points
        model = smf.ols('net_generation_mw ~ gross_generation_mw', data=df).fit()

        # find and remove any outliers recursively up to two times
        outliers = model.outlier_test()
        if abs(outliers['student_resid']).max() > 3:
            # remove any outlier values
            df = df[~df.index.isin(outliers[abs(outliers['student_resid']) > 3].index)]

            # get a linear model of the corrected data
            model = smf.ols(
                    'net_generation_mw ~ gross_generation_mw', data=df).fit()
            
            outliers = model.outlier_test()

            # perform this removal one more time in case any outliers were masked by the first outlier(s)
            if abs(outliers['student_resid']).max() > 3:
                # remove any outlier values
                df = df[~df.index.isin(outliers[abs(outliers['student_resid']) > 3].index)]

                # get a linear model of the corrected data
                try:
                    model = smf.ols(
                            'net_generation_mw ~ gross_generation_mw', data=df).fit()
                except ValueError:
                    pass
        slope = model.params[1]
        intercept = model.params[0]
        rsquared = model.rsquared
        rsquared_adj = model.rsquared_adj
        number_observations = model.nobs

    return slope, intercept, rsquared, rsquared_adj, number_observations

In [ ]:
# calculate the ratio for each plant and create a dataframe
gtn_regression = gen_data.dropna().groupby(['plant_id_eia','subplant_id']).apply(model_gross_to_net)
gtn_regression = pd.DataFrame(gtn_regression.tolist(), index=gtn_regression.index, columns=['slope','intercept', 'rsquared','rsquared_adj','observations']).reset_index()
gtn_regression

In [ ]:
gtn_regression[(gtn_regression['rsquared_adj'] < 0.9) & (gtn_regression['rsquared_adj'] > 0.8)]

In [ ]:
# remove results with a negative rsquared adjusted
gtn_regression = gtn_regression[gtn_regression['rsquared_adj'] > 0]

In [ ]:
gtn_regression.to_csv('../data/outputs/gross_to_net_regression.csv', index=False)

In [ ]:
gtn_regression.mean().round(2)

# Test Applying the values to the hourly CEMS data

In [ ]:
id = 3
test_cems = cems[cems['plant_id_eia'] == id]
test_cems

In [ ]:
test_cems = test_cems.merge(gtn_regression[['plant_id_eia','subplant_id','slope','intercept']], how='left', on=['plant_id_eia','subplant_id'])
# divide the house load by the number of hours in each month
#test_cems['intercept'] = test_cems['intercept'] / (test_cems.report_date.dt.daysinmonth * 24)
# divide the house load by the number of units in each subplant
units_in_subplant = subplant_crosswalk.groupby(['plant_id_eia','subplant_id'])['unitid'].count().reset_index().rename(columns={'unitid':'units_in_subplant'})
test_cems = test_cems.merge(units_in_subplant, how='left', on=['plant_id_eia','subplant_id'])
test_cems['intercept'] = test_cems['intercept'] / test_cems['units_in_subplant']
# calculate net generation
test_cems['net_generation_mwh'] = test_cems['gross_generation_mwh'] * test_cems['slope'] + test_cems['intercept']
test_cems

In [ ]:
px.line(test_cems, x='operating_datetime_utc', y=['gross_generation_mwh','net_generation_mwh'], facet_col='unitid')

In [ ]:
test_cems.groupby(['plant_id_eia','subplant_id','report_date'])['net_generation_mwh'].sum()

In [ ]:
gf_to_compare = gen_fuel_allocated[gen_fuel_allocated['plant_id_eia'] == 3]
gf_to_compare = gf_to_compare.groupby(['plant_id_eia','subplant_id','report_date'])['net_generation_mwh'].sum()
gf_to_compare

# Old Code for reference

In [ ]:


def convert_gross_to_net_generation(cems, gen_fuel_allocated):
    """
    Converts hourly gross generation in CEMS to hourly net generation by calculating a gross to net generation ratio
    Inputs:

    Returns: 
        cems df with an added column for net_generation_mwh and a column indicated the method used to calculate net generation
    """

    # add a placeholder column that assumes a 1:1 gross to net generation ratio
    # if for some reason we are not able to calculate a gross to net generation ratio, this will be used as the default assumption
    cems['net_generation_mwh'] = cems['gross_generation_mwh']

    # load the allocated eia data for each month where there is corresponding cems data
    eia_plant_month_net_gen = gen_fuel_allocated[(gen_fuel_allocated['data_source'] == 'cems') & ~(gen_fuel_allocated['net_generation_mwh'].isna())]
    # aggregate at the plant month level
    eia_plant_month_net_gen = eia_plant_month_net_gen.groupby(['plant_id_eia','report_date']).sum()['net_generation_mwh'].reset_index()

    # calculate the total gross generation for each plant month in cems
    cems_plant_month_gross_gen = cems.groupby(['plant_id_eia','report_date']).sum()['gross_generation_mwh'].reset_index()

    # merge the net generation data into the gross generation data
    monthly_gtn_ratio = cems_plant_month_gross_gen.merge(eia_plant_month_net_gen, how='left', on=['plant_id_eia','report_date'])

    # calculate the gtn
    monthly_gtn_ratio['gross_to_net_ratio'] = monthly_gtn_ratio['net_generation_mwh'] / monthly_gtn_ratio['gross_generation_mwh']

    # only keep values where the monthly ratio is greater than zero
    monthly_gtn_ratio.loc[(monthly_gtn_ratio['gross_to_net_ratio'] < 0), 'gross_to_net_ratio'] = np.NaN

    # Set up the regression analysis for missing values

    # only keep values where there are not missing values
    gtn_regression = monthly_gtn_ratio.copy()[~(monthly_gtn_ratio['gross_to_net_ratio'].isna())]
    # calculate the ratio for each plant and create a dataframe
    gtn_regression = gtn_regression.groupby('plant_id_eia').apply(model_gross_to_net)
    gtn_regression = pd.DataFrame(gtn_regression.tolist(), index=gtn_regression.index, columns=['gtn_linear', 'rsquared','rsquared_adj','observations']).reset_index()
    # only keep the results with adjusted rsquared values greater than 0.70
    gtn_regression = gtn_regression[gtn_regression['rsquared_adj'] >= 0.7]

    # merge in regression results
    monthly_gtn_ratio = monthly_gtn_ratio.merge(gtn_regression[['plant_id_eia','gtn_linear']], how='left', on='plant_id_eia')

    # add a status column for how the net generation was calculated 
    monthly_gtn_ratio['net_gen_method'] = 'monthly_ratio'
    monthly_gtn_ratio.loc[(monthly_gtn_ratio['gross_to_net_ratio'].isna()) & ~(monthly_gtn_ratio['gtn_linear'].isna()), 'net_gen_method'] = 'annual_regression'
    monthly_gtn_ratio.loc[(monthly_gtn_ratio['gross_to_net_ratio'].isna()) & (monthly_gtn_ratio['gtn_linear'].isna()), 'net_gen_method'] = 'net_equals_gross'

    # fill missing values using the ratio from the regression results
    monthly_gtn_ratio['gross_to_net_ratio'] = monthly_gtn_ratio['gross_to_net_ratio'].fillna(monthly_gtn_ratio['gtn_linear'])

    # merge the gtn ratio into the cems data
    cems = cems.merge(monthly_gtn_ratio[['plant_id_eia','report_date','gross_to_net_ratio','net_gen_method']], how='left', on=['plant_id_eia','report_date'])
    # calculate hourly net generation
    cems['net_generation_mwh_calculated'] = cems['gross_generation_mwh'] * cems['gross_to_net_ratio']
    
    # update the net generation column using the calculated values
    cems['net_generation_mwh'].update(cems['net_generation_mwh_calculated'])

    # update the method column to indicate which used the default assumption
    cems['net_gen_method'] = cems['net_gen_method'].fillna('net_equals_gross')

    # drop the calculated column
    cems = cems.drop(columns=['net_generation_mwh_calculated'])

    return cems

In [ ]:
# TESTING

# 1a: Try monthly GTN by generator
# load the allocated eia data for each month where there is corresponding cems data
eia_gen_month_net_gen = gen_fuel_allocated[(gen_fuel_allocated['data_source'] == 'cems') & ~(gen_fuel_allocated['net_generation_mwh'].isna())]
# aggregate at the generator month level
eia_gen_month_net_gen = eia_gen_month_net_gen.groupby(['plant_id_eia','generator_id','report_date']).sum()['net_generation_mwh'].reset_index()

# match unit id to generator id
cems_gen_month_gross_gen = data_cleaning.crosswalk_epa_unit_to_eia_generator_id(cems, unique_gen_match=True)
# drop any observations where there is not a match to a generator id
cems_gen_month_gross_gen = cems_gen_month_gross_gen[~cems_gen_month_gross_gen['generator_id'].isna()]

# calculate the total gross generation for each generator month in cems
cems_gen_month_gross_gen = cems_gen_month_gross_gen.groupby(['plant_id_eia','generator_id','report_date']).sum()['gross_generation_mwh'].reset_index()

# merge the net generation data into the gross generation data
monthly_gtn_ratio = cems_gen_month_gross_gen.merge(eia_gen_month_net_gen, how='left', on=['plant_id_eia','generator_id','report_date'])

# calculate the gtn
monthly_gtn_ratio['gtn_ratio_gen_month'] = monthly_gtn_ratio['net_generation_mwh'] / monthly_gtn_ratio['gross_generation_mwh']

#### Calculate annual values by gen

# identify whether any individual month GTNs are very large
annual_gtn_ratio = monthly_gtn_ratio.groupby(['plant_id_eia','generator_id']).max()['gtn_ratio_gen_month'].reset_index()
# only keep values for which the maximum GTN values are unrealistic
annual_gtn_ratio = annual_gtn_ratio[(annual_gtn_ratio['gtn_ratio_gen_month'] > 1.1) | (annual_gtn_ratio['gtn_ratio_gen_month'] < 0)]

# calculate annual net gen values
eia_gen_year_net_gen = eia_gen_month_net_gen.groupby(['plant_id_eia','generator_id']).sum().reset_index()
cems_gen_year_gross_gen = cems_gen_month_gross_gen.groupby(['plant_id_eia','generator_id']).sum().reset_index()

# merge in the annual gross generation and net generation data
annual_gtn_ratio = annual_gtn_ratio.merge(cems_gen_year_gross_gen, how='left', on=['plant_id_eia','generator_id'])
annual_gtn_ratio = annual_gtn_ratio.merge(eia_gen_year_net_gen, how='left', on=['plant_id_eia','generator_id'])

# calculate the gtn
annual_gtn_ratio['gtn_ratio_gen_year'] = annual_gtn_ratio['net_generation_mwh'] / annual_gtn_ratio['gross_generation_mwh']

# merge this back into the monthly data
monthly_gtn_ratio = monthly_gtn_ratio.merge(annual_gtn_ratio[['plant_id_eia','generator_id','gtn_ratio_gen_year']], how='left', on=['plant_id_eia','generator_id'])

# NOTE
# In some cases, using the annual gtn value may lead to overestimating monthly generation
# it might depend on how the generation is getting allocated

monthly_gtn_ratio[monthly_gtn_ratio['plant_id_eia'] == 61242]